### EXAMPLE

This notebook reads the content of the extract to plot the network on the Grenoble metropolis area.

In [1]:
from ipyleaflet import Map, LayersControl, LayerGroup, GeoJSON, LegendControl, FullScreenControl, basemaps, WKTLayer
import json
from tqdm.auto import tqdm, trange
from matplotlib import colormaps
from matplotlib.colors import ListedColormap
import numpy as np
from shapely.geometry import shape, Polygon
from collections import defaultdict

In [2]:
# metropolis boundaries
with open("./GRA_metropole.geojson", 'r') as f:
    metropole: Polygon = shape(json.load(f)["features"][0]["geometry"])

In [3]:
def clean(feature: dict) -> dict:
    """
    Clean GeoJSON feature
    """
    feature["properties"] = {p:v for p,v in feature["properties"].items() if v is not None}
    if 'way' in feature['properties'].keys():
        del feature['properties']['way']
    return feature

def rgb_to_hex(palette: tuple[int, int, int]) -> str:
    """
    Convert RGB to hexadecimal
    """
    palette = [int(255*x) for x in palette]
    return '#{:02x}{:02x}{:02x}'. format(*palette)

def parse_line(line: str) -> str:
    """
    Clean a .csv line to read the content
    """
    return line.replace("\"\"","\"").replace('\n', '')

In [4]:
result_file = './extract/extract.txt'
geo_features = defaultdict(list)

# Read the extract file
with open(result_file, 'r', encoding='ISO-8859-1') as f:
    lines = f.readlines()

for i in trange(len(lines) // 2):
    geodata = parse_line(lines[2*i+1])
    tag = parse_line(lines[2*i])

    geo_line = json.loads(geodata[1:-1]) # remove the "" around the dict and parse it

    # If there is no feature in the geojson, skip it
    if geo_line['features'] is None: 
        continue
    
    # keep the feature if it is spatially related to the metropolis area
    feature_list = geo_line["features"]
    res = []
    for feature in feature_list:
        poly = shape(feature['geometry'])
        if  metropole.intersects(poly):
            res.append(clean(feature))
    geo_line["features"] = res

    # create a new GeoJSON feature in leaflet and add it to a new layer
    geo_features[tag].append(geo_line)

  0%|          | 0/56 [00:00<?, ?it/s]

In [5]:
# prepare the map
center = metropole.centroid.coords[0][::-1] 
m = Map(basemap=basemaps.Esri.WorldTopoMap, center=center)
b = metropole.bounds
bounds = [[b[1], b[0]], [b[3], b[2]]]
m.fit_bounds(bounds)
legend = LegendControl(legend={},name="Legend", position="bottomright")
m.add_control(legend)

# add Metropole boundaries
m.add_layer(LayerGroup(layers=[WKTLayer(wkt_string=metropole.wkt)], name="Metropole"))

# setup color scale
n = len(geo_features.keys())
colormap = colormaps.get_cmap("hsv")
aux_array = np.linspace(0, 1, n)
color_vector = ListedColormap(colormap(aux_array))
roadPalette = {}
for i in range(n):
    roadPalette[sorted(list(geo_features.keys()))[i]] = color_vector.colors[i]


# add all geodata
for key, geojson_list in tqdm(sorted(list(geo_features.items()))):

    color = rgb_to_hex(roadPalette[key])
    legend.add_legend_element(key, color)
    # create a new GeoJSON feature in leaflet and add it to a new layer
    layer_group = LayerGroup(layers=[GeoJSON(data=geo_line, style={'color': color}) for geo_line in geojson_list], name=key)
    m.add_layer(layer_group)

# add controls
control = LayersControl(position='topleft')
m.add_control(control)
m.add_control(FullScreenControl())

# show the map
m

  0%|          | 0/48 [00:00<?, ?it/s]

Map(center=[45.141861063952376, 5.718250822399647], controls=(ZoomControl(options=['position', 'zoom_in_text',…